# Inteligentni sistemi genetsko programiranje za simbolično regresijo
**Genetski algoritm napisan v Pythonu**

Avtorja:

Luka Premuš, Domen Beden


### Knjižnice:

In [18]:
import pandas as pd
from r import *
from r2 import *
import json
import math
from sympy import simplify
import matplotlib.pyplot as plt


data = pd.read_csv("./dataset.csv")
formule = data['Equation']


## Generiranje populacije

Za začetek delovanja genetskega algoritma je potrebna začetna populacija. V našem primeru je primerek iz populacije aritmetični izraz. 

Pri generiranju aritmetičnih izrazov naključno izberamo operacije in vrednosti v izrazih. 

**Možne operacije**: seštevanje (+), odštevanje (-), množenje (*), deljenje (/) in potenciranje (**)

**Možne vrednosti**: števila od 1 do 9, neodvisna spremenljivka $x$, naravni logaritem od števil 1 do 9, naravni logaritem od spremenljivke $x$ sinus od spremenljivke $x$, kosinus od spremenljivke $x$, od števil 1 do 9 in kosinus od števil 1 do 9. V naboru vrednosti, ki jih izbira algoritem pri izdelavi artimetične enačbe, se izbor "števila od 1 do 9" ponovi večkrat, saj s tem imajo večjo verjetnost, da so izbrane številke pri generaciji. S tem se izogneva izdelavi pretirane kompleksnosti enačb. 

Nato naključno generiramo število vrednosti, ki se nahaja med naravnim številom podanim z argumentom *numTerms* in *numTerms + 2*.

Funkcija *generate_expression* poleg tega da naključno izbira operacije in vrednosti, poskrbi tudi, da se v izrazu nahaja vsaj ena neodivsna spremenljivka $x$.

Funkcija se strogo izogiba enačbam, ki vsebujejo $x^{x}$. Te enčbe se izkažejo za težko obvladljive in hitro pride do **preliva** (ang. *Overflow*). 

In [29]:
def generate_expression(numTerms, x_index):
    operators = ["+", "-", "*","/", "**"]
    vals = [str(random.randint(1, 9)),str(random.randint(1, 9)),
            str(random.randint(1, 9)),str(random.randint(1, 9)),
            str(random.randint(1, 9)),
              "x", f"log{str(random.randint(1,9))}",
              "logx","sinx","cosx",
              f"sin{str(random.randint(1,9))}",
              f"cos{str(random.randint(1,9))}"]
    vals2 = str(random.randint(1, 9))
    
    num_terms = random.randint(numTerms, 2+numTerms)
    
    x_index %= numTerms
    correct = False
    while not correct:
        correct = True
        terms = []
        del terms[:]
        x = 0
        for i in range(num_terms):
            
            op = random.choice(operators)
            num = random.choice(vals)

            if x_index == i:
                if i > 0 and terms[i-1] == "**":
                    correct = False
                    break     #da ne dobimo neki**x
                else:
                    num = "x"
                    x += 1
            else:
                if i > 0 and terms[i-1] == "**":
                    
                    if num == "x":
                        correct = False
                        break
            terms.append(num)
            terms.append(op)

        terms.append(vals2)
        expression = "".join(terms)

    return expression

Primeri 15 generiranih enačb z minimalno petimi vrednostmi:

In [30]:
for i in range(15):
    equation = generate_expression(5, i)
    print(f"{i}: {equation}")


0: x*sin5/logx-cos6-sinx+cos6*3
1: cosx**x-sin4*logx/sinx**sin4+3
2: logx*cosx-x/1-sin4/2
3: 5/sin7+sin7/x/5-cos5*x+5
4: cosx+log6+sinx*cos9**x*sin7+cos9+3
5: x/cosx**sin5/8-cos9-logx*2
6: 8-x-x**x/2+6/1
7: log8-cos9**x*logx-sinx/1
8: cos6+cosx-sin4/x+cos6-7+6
9: 2*log8/sin5*4**x*4
10: x**logx*4/8**sinx**1/4**6
11: 5+x*6+5**log2*cosx-6
12: cosx*9**x*6/log3-logx**1
13: log8/x*cos8**x+sin9+logx/7
14: cosx**6*8+1-x-2+6


## Podatkovana strkutura podatkov

Kot je bilo podano v navodilih sva za podatkovno strkturo izbrala drevo. 
Drevo se gradi na podlagi prioritet v operacijah. Funkcija `fix2` preobrazi aritmetični izraz v ekvivalentnega, vendar glede na prioritete postavi oklepaje. S tem si olajšava gradnjo drevesa. 


In [ ]:
def fix2(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "^":
            if i+1 < len(tokens) - 1 and tokens[i+1] == "(":
                #while
                n = i
                while tokens[n] != ")":
                    n += 1
                tokens.insert(n, ")")
            else:
                tokens.insert(i + 2, ")")
            #sam se v drugo smer ni b lema
            if tokens[i-1] == ")":
                m = i
                while tokens[m] != "(":
                    m -= 1
                tokens.insert(m, "(")
            else:
                tokens.insert(i - 1, "(")
            i += 1

        i += 1


    i = 0
    while i < len(tokens):
        if tokens[i] in "*/":
            if i+1 < len(tokens) - 1 and tokens[i+1] == "(":
                #while
                n = i
                while tokens[n] != ")":
                    n += 1
                tokens.insert(n, ")")
            else:
                tokens.insert(i + 2, ")")
            #sam se v drugo smer
            if tokens[i-1] == ")":
                m = i
                while tokens[m] != "(":
                    m -= 1
                tokens.insert(m, "(")
            else:
                tokens.insert(i - 1, "(")

            i += 1

        i += 1
    return tokens

Gradnja drevesa poteka v funkciji `build_tree`. Drevo ima v listih vrednosti, v vejah pa operacije. Zgrajeno je tako, da operacija, ki ima večjo prioriteto je globje kot pa operacija z manjšo. S tem poskrbiva, da je izračun drevesa pravilen, ko vstaviva notri vrednost.

In [ ]:
def build_tree(tokens):

    stack = []
    tree = None

    for token in tokens:
        if token not in "*)^/+-(*":
            if token in "sxcxlxx":
                node = Node(token)
            else:
                if not token[0].isnumeric():
                    node = Node(token)
                else:
                    node = Node(int(token))
            stack.append(node)
        elif token == "(":

            stack.append(token)

        elif token in "+-*/^":
            right = stack.pop()
            left = stack.pop()

            
            node = Node(token)
            node.left = left
            node.right = right
            stack.append(node)
        
        elif token == ")":
            node = stack.pop()
            
            lastOp = node
            while node != "(":

                node = stack.pop()
            stack.append(lastOp)
    tree = stack.pop()
    return tree

Izračn drevesa poteka v funkciji `calculate_tree`. Na vhodu dobi koren drevesa, in vrednost, ki jo vsatavi namesto spremenljivke $x$. 

In [1]:
def calculate_tree(node, x):
    if type(node.value) == int:
        return int(node.value)
    if len(node.value) > 1 and node.value[0] == '-':
        return int(node.value)
    # sinus
    if node.value[0] == 's':
        if (node.value[1] == 'x'):
            return math.sin(x)
        value = node.value[1:]
        return math.sin(int(value))
    
    # cosinus
    if node.value[0] == 'c':
        if (node.value[1] == 'x'):
            return math.cos(x)
        value = node.value[1:]
        return math.cos(int(value))
    
    # logaritem
    if node.value[0] == 'l':
        if (node.value[1] == 'x'):
            return math.log(x)
        value = node.value[1:]
        return math.log(int(value))

    elif node.value == 'x':
        return x
    else:
        if node.value == '+':
            return calculate_tree(node.left,x) + calculate_tree(node.right,x)
        elif node.value == '-':
            return calculate_tree(node.left, x) - calculate_tree(node.right, x)
        elif node.value == '*':
            return calculate_tree(node.left, x) * calculate_tree(node.right, x)
        elif node.value == '/':
            x2 = calculate_tree(node.right, x)
            x1 = calculate_tree(node.left, x)

            if x2 == 0:
                return x1 // 1
            return x1 // x2

        elif node.value == '^':
            x1 = calculate_tree(node.left, x)
            if x1 < 0.0001:
                return 0
            
            x2 = calculate_tree(node.right, x)
            if x2 < 0.0001:
                return 0

            try:
                x3 = round(x1 ** x2)
                if x3 > 10_000_000:
                    return 10_000_000
                return round(x3,1)
            except OverflowError as e:
                return 10_000_000

# Fitness funkcije

Fitness funkcije so uporabljene za ocenjevanje primerkov. S temi ocenami lahko določimo kateri primeri se bodo ohranili in bodo uporabljeni za reprodukcijo. 

Najna fitness funkcija `fitness_MSE` deluje na principu `povprečju kvadratov napak`(ang. mean square error). Poleg tega pa upošteva še dolžino enačbe.

Funkcija `fitness_MSE` dobi na vhod predvideno enačbo, ki ji mora izračunati fitness, tabelo podatkov s pravilnimi enačbami, vhodi in izhodi, ter še index enačbe, ki jo trenutno iščemo.

Izračuna razliko med narvanim logaritmom pravilne vrednosti in naravnim logaritmom izračunano vrednostjo predvidene enačbe. Nato to kvadrira in doda na seznam rezultatov. Ko to izračunamo za vse vhode, seštejemo povprečne kvadratne napake in jih delimo s količino le teh. 

Nato to poračunamo še kazen glede na dolžino enačbe (krajše enačbe so manj kaznovane kot daljše), vse skupaj zaokrožimo na 5 decimalk in vrnemo.

In [ ]:
def fitness_MSE(guess_eq, df, index):
    row = df.iloc[index]
    Xs = json.loads(row['Xs'])
    
    Ys = json.loads(row['Ys'])
    real_eq = row['Equation']

    tokens = parse_equation1(guess_eq)

    fix_tokens = fix2(tokens)
    sorted_tokens = sort_tokens(fix_tokens)
    tree = build_tree(sorted_tokens)
    fitnessVal = 0

    MSE_error = []
    for i in range(len(Xs)):
        rez = (calculate_tree(tree, int(Xs[i])))
        if (rez > 0):
            rez = math.log(rez)
        real_val = int((Ys[i]))
        if real_val > 0:
            real_val=math.log(real_val)

        sub = real_val - rez
        sub = sub
        try:
            MSE = (sub)**2
            # if MSE > 0:
            #     MSE = math.abs(MSE)
        except (OverflowError):
            return np.inf

        MSE_error.append(MSE)
        penalty = len(guess_eq)*0.1

    
    return round(sum(MSE_error)/len(MSE_error)+penalty,5)

## Mutacije

Mutacije se se izvajajo na dva načina tako da mutirajo vrednosti in tako da mutirajo operacije.

V funkciji `mutate_value` mutirava vrednosti. To počneva tako da z verjetnostjo $25$% zamenjava številko z novo vrednostjo iz nabora vrednosti.

V funkciji `mutate_operation` mutirava operacije. To počneva tako da z verjetnostjo $25$% zamenjava operacijo z novo operacijo iz nabora operacij.

*Nabor operacij*: seštevanje (+), odštevanje (-), množenje (*), deljenje (/), potenciranjej (^)


In [ ]:
def mutate_value(expression):
    tokens = parse_equation1(expression)
    for i,c in enumerate(tokens):
        # print(c)
        if c not in "sinxcoslogxx*/()-+" and c != "x":
            mutateBool = [True, False, False, False]
            if random.choice(mutateBool):
                rand_vals = [str(random.randint(-9, -1)),
                              str(random.randint(1, 9)), f"log{str(random.randint(1,9))}","logx","sinx","cosx",
                              f"sin{str(random.randint(1,9))}",f"cos{str(random.randint(1,9))}"]
                new_val = random.choice(rand_vals)
                tokens[i] = new_val
    return tokens


def mutate_operation(expression):
    tokens = parse_equation1(expression)
    for i,c in enumerate(tokens):
        # print(c)
        if c in "*/-+^":
            mutateBool = [True, False, False, False]
            if random.choice(mutateBool):
                rand_op = ["+", "-", "*","/", "**"]
                new_val = random.choice(rand_op)
                tokens[i] = new_val
    return tokens

## Križanje

## Izvedba genetskega algoritma

In [28]:
equationIndex = 10

mutation_choices = [[1,1,1,2,2,3,2,3,3,3,3,3,3,3,3,3,3,3,3,3], [1,1,1,2,2,2], [3], [2], [1]]

def ga(N, gen, equationIndex):
    
    prevN = N
    best_of_gens = {}
    for i in range(gen):
        # print(i+1)
        results = {}
        N = prevN

        for i in range(N):
            expression = generate_expression(5, i)
            fitness = fitness_MSE(expression, data, equationIndex)
            results[fitness] = expression
    
        N = N // 2
        
        result = sort_dict(results)
        new_dict = dict(list(result.items())[:N])

        # [1,1,3,3,3,3,3,3,3,3,3,3,3,3,3]
        # [0,0,0,0,0,0,1, 1,1, 2,3,3,3,3, 3]
        while N > 1:
            for ele in new_dict:
                mutation_choice = random.choice([1,1,1,2,2,3,2,3,3,3,3,3,3,3,3,3,3,3,3,3])

                if mutation_choice == 1:
                    expression = arr_to_string(mutate_value(result[ele]))   
                    fitness = fitness_MSE(expression, data, equationIndex)
                    
                    result[fitness] = expression
                if mutation_choice == 2:
                    expression = arr_to_string(mutate_operation(result[ele]))

                    fitness = (fitness_MSE(expression, data, equationIndex))
                    result[fitness] = expression

                if mutation_choice == 3:
                    tree1, tree2 = pick2trees(new_dict)
                    tree1, tree2 = crossover(tree1, tree2)

                    #ce ni x v enacbi ponavlja 
                    while(check_if_x(tree1) == False) or (check_if_x(tree2) == False):
                        tree1_str = print_infix_parenthesis(tree1)

                        tree2_str = print_infix_parenthesis(tree2)
                        # print(f"tree1: {tree1_str}, tree2: {tree2_str}")
                        tree1,tree2 = crossover(tree1,tree2)
                        

                    tree1_str = infixString(tree1)
                    tree2_str = infixString(tree2)
                    
                    fitness1 = fitness_MSE(tree1_str, data, equationIndex)
                    fitness2 = fitness_MSE(tree2_str, data, equationIndex)


                    result[fitness1] = tree1_str
                    result[fitness2] = tree2_str

                    

            N = N // 2
            result = sort_dict(results)
            new_dict = dict(list(result.items())[:N]) 
        
        #vrne najboljsega iz generacije
        val = list(sort_dict(results))[0]
        # val = [simplify(results[]) : e in ]
        # best_of_gens[val] = (results[val])
        best_of_gens[val] = simplify(results[val])

        # print("best of gen: " + results[val] + " " + str(val))

    return best_of_gens

k = ga(100, 100, equationIndex)
k = sort_dict(k)


val = list(k)[0]

row = data.iloc[equationIndex]
Xs = json.loads(row['Xs'])

Ys = json.loads(row['Ys'])
real_eq = row['Equation']
print("---------------------")

print(real_eq)
print("---------------------")

print(k[val], val)




---------------------
(((x - 2) + 4) * 5)
---------------------
5*x + 9 - 3/3**(5**sin3) 0.00016


In [ ]:
function_accuracy = []

for eqIndex in range(52, len(data)-40):
    
    k = ga(1000, 20, eqIndex)
    k = sort_dict(k)

    val = list(k)[0]

    row = data.iloc[eqIndex]
    Xs = json.loads(row['Xs'])

    Ys = json.loads(row['Ys'])
    real_eq = row['Equation']
    print("---------------------")
    print(f"{eqIndex} - real eq: " + real_eq)
    print("=====================")
    print(k[val], val)
    print("---------------------")
    function_accuracy.append(val)



---------------------
52 - real eq:  -4*x**5 + 1*x**3 + -2*x**2 + -1*x + -1
-5**logx*x**5 + x + 10 3.9063166127738356e+19
---------------------
---------------------
53 - real eq:  -4*x + +2
cos9 - sin1 - 4*x + 3 0.56641
---------------------
---------------------
54 - real eq:  -4*x**4 + -2*x**3 + 1*x + -2
-x**5/5 + x + 6 3310596063012415.5
---------------------
---------------------
55 - real eq:  4*x**2 + -2*x + +2
-3*cos8*cosx + 4*x**2 0.00082
---------------------
---------------------
56 - real eq:  -4*x**4 + 1*x + -2
cosx - 38880*x**2 3577366395614657.5
---------------------
---------------------
57 - real eq:  1*x**3 + -2*x**2 + -1*x + 4
-cos9*x**3 + 1 0.04038
---------------------


In [ ]:
values = function_accuracy.copy()

values = [100 if v > 100 else v for v in values]

xs = np.arange(98)
plt.scatter(xs,values,color='red',marker='o')
plt.plot(xs,values,color='blue')


In [ ]:
function_accuracy1 = []
for eqIndex in range(0, len(data)):
    
    k = ga(200, 40, eqIndex)
    k = sort_dict(k)

    val = list(k)[0]

    function_accuracy1.append(val)

In [ ]:
values = function_accuracy1.copy()

values = [100 if v > 100 else v for v in values]

xs = np.arange(98)
plt.scatter(xs,values,color='red',marker='o')
plt.plot(xs,values,color='blue')

In [ ]:
function_accuracy2 = []
for eqIndex in range(0, len(data)):
    
    k = ga(500, 500, eqIndex)
    k = sort_dict(k)

    val = list(k)[0]

    function_accuracy2.append(val)

In [ ]:
values = function_accuracy2.copy()

values = [100 if v > 100 else v for v in values]

xs = np.arange(98)
plt.scatter(xs,values,color='red',marker='o')
plt.plot(xs,values,color='blue')

In [ ]:
function_accuracy3 = []
for eqIndex in range(0, len(data)):
    
    k = ga(100, 100, eqIndex)
    k = sort_dict(k)

    val = list(k)[0]

    function_accuracy3.append(val)

In [ ]:
values = function_accuracy3.copy()

values = [100 if v > 100 else v for v in values]

xs = np.arange(98)
plt.scatter(xs,values,color='red',marker='o')
plt.plot(xs,values,color='blue')

In [ ]:
function_accuracy4 = []
for eqIndex in range(0, len(data)):
    
    k = ga(800, 50, eqIndex)
    k = sort_dict(k)

    val = list(k)[0]

    function_accuracy4.append(val)

In [ ]:
values = function_accuracy4.copy()

values = [100 if v > 100 else v for v in values]

xs = np.arange(98)
plt.scatter(xs,values,color='red',marker='o')
plt.plot(xs,values,color='blue')

In [ ]:
function_accuracy5 = []
for eqIndex in range(0, len(data)):
    
    k = ga(200, 300, eqIndex)
    k = sort_dict(k)

    val = list(k)[0]

    function_accuracy5.append(val)

In [ ]:
values = function_accuracy5.copy()

values = [100 if v > 100 else v for v in values]

xs = np.arange(98)
plt.scatter(xs,values,color='red',marker='o')
plt.plot(xs,values,color='blue')